In [14]:
from scipy.stats import norm
import numpy as np
import yfinance as yf
import pandas as pd

from datetime import datetime

class BsOption:
    def __init__(self, S, K, T, r, sigma, q=0):
        self.S = S
        self.K = K
        self.T = T
        self.r = r 
        self.sigma = sigma
        self.q = q
        
    
    @staticmethod
    def N(x):
        return norm.cdf(x)
    
    @property
    def params(self):
        return {'S': self.S, 
                'K': self.K, 
                'T': self.T, 
                'r':self.r,
                'q':self.q,
                'sigma':self.sigma}
    
    def d1(self):
        return (np.log(self.S/self.K) + (self.r -self.q + self.sigma**2/2)*self.T) \
                                / (self.sigma*np.sqrt(self.T))
    
    def d2(self):
        return self.d1() - self.sigma*np.sqrt(self.T)
    
    def _call_value(self):
        return self.S*np.exp(-self.q*self.T)*self.N(self.d1()) - \
                    self.K*np.exp(-self.r*self.T) * self.N(self.d2())
                    
    def _put_value(self):
        return self.K*np.exp(-self.r*self.T) * self.N(-self.d2()) -\
                self.S*np.exp(-self.q*self.T)*self.N(-self.d1())
    
    def price(self):
            #return self._call_value()
            return self._call_value(),self._put_value()
        
def GetData(stk,start_date,end_date):
    df=yf.download(stk, start_date, end_date,interval='1d').dropna()
    return(df)
def ModData(df): 
    df['dow']=df.index.dayofweek
    df['woy']=df.index.isocalendar().week+df.index.isocalendar().year*100
    #df=df[df['dow']!=3]
    #df=df[df['dow']!=2]
    #df=df[df['dow']!=1]
    #df=df[df['dow']!=0]
    df=df.groupby(['woy']).agg({'Open':'first','Close':'last'})
    df['K']=df.Open.round(0)
    return(df)

df=GetData('XBI','2020-01-01','2022-06-30')
df=ModData(df)

IV=0.5
Days=5
df['C1']=BsOption(df.Open, df.K, Days/365, 0.03, IV).price()[0]
df['C2']=BsOption(df.Close, df.K, 0/365, 0.03, IV).price()[0]
df['P1']=BsOption(df.Open, df.K, Days/365, 0.03, IV).price()[1]
df['P2']=BsOption(df.Close, df.K, 0/365, 0.03, IV).price()[1]
df['T1']=df.C1+df.P1
df['T2']=df.C2+df.P2
df['PL']=df.T1-df.T2
df['LogR']=np.log((df.Open+df.PL)/df.Open)
print(df.tail())

print(np.sum(df.LogR))
print(np.sum(df[df.LogR>0].LogR))
print(np.sum(df[df.LogR<=0].LogR))
print(len(df[df.LogR>0].LogR))
print(len(df.LogR))

#K = 65
#r = 0.03
#T = 1/365
#sigma = 0.75
#S = 67.61
#res=(BsOption(S, K, T, r, sigma).price())
#print(res[1])
#df.to_csv('opt_yield.csv')


[*********************100%***********************]  1 of 1 completed
             Open      Close     K        C1        C2        P1        P2  \
woy                                                                          
202222  70.790001  71.209999  71.0  1.565720  0.209999  1.746547  0.000000   
202223  72.180000  67.029999  72.0  1.789319  0.000000  1.579736  4.970001   
202224  64.970001  67.610001  65.0  1.514988  2.610001  1.518280  0.000000   
202225  68.660004  77.059998  69.0  1.455351  8.059998  1.766997  0.000000   
202226  77.019997  74.800003  77.0  1.823210  0.000000  1.771576  2.199997   

              T1        T2        PL      LogR  
woy                                             
202222  3.312266  0.209999  3.102267  0.042890  
202223  3.369055  4.970001 -1.600946 -0.022430  
202224  3.033267  2.610001  0.423267  0.006494  
202225  3.222348  8.059998 -4.837649 -0.073063  
202226  3.594786  2.199997  1.394789  0.017947  
1.3393541933158608
2.37389820118669
-1.03